# Data Cleaning Appendix

## Data collection and cleaning

Our team was unable to find a dataset that contained pricing data for the S&P 500 over the last 10 years. Therefore, in order to gather this data into a workable dataset, we utilized a list of the S&P 500 companies along with the Yahoo Finance API to create our own dataset.

Source data:
- Data of current S&P 500 Company from https://github.com/datasets/s-and-p-500-companies/blob/master/data/constituents.csv
- Yahoo Finance API: http://theautomatic.net/yahoo_fin-documentation/

These files can be downloaded in the Google Drive folder https://drive.google.com/drive/folders/1I_hP_G51eLKYcYOwQKrcTGOimkLhTL52?usp=sharing

In [33]:
import pandas as pd

In [ ]:
# 1. Convert data from S&P 500 Companies list into text file with tickers of each company
spList = pd.read_csv('Source Data/constituents.csv')
spListTick = spList["Symbol"].tolist()
spListTick.append('SPY') # adding SPY into list
textfile = open("companies.txt", "w")
for element in spListTick:
    textfile.write(element + " ")
textfile.close()


We first downloaded a csv file on the current S&P 500 companies and their associated sectors from https://github.com/datasets/s-and-p-500-companies/blob/master/data/constituents.csv. We converted the list of companies to a txt file to be used to later call the Yahoo Finance API. We also wrote the S&P 500 Index into the list as well (with the ticker being SPY) in order to gain some more data on the overall market's performance.

In [17]:
# 1. Convert data from S&P 500 Companies list into text file with tickers of each company
spList = pd.read_csv('Source Data/constituents.csv')
spListTick = spList["Symbol"].tolist()
spListTick.append('SPY') # adding SPY into list
textfile = open("companies.txt", "w")
for element in spListTick:
    textfile.write(element + " ")
textfile.close()

We then scraped data from the Yahoo Finance API.

In [ ]:
# install yahoo finance API
import sys
!pip3 install yahoo-fin

In [ ]:
# 2. 
import yahoo_fin.stock_info as si
import pandas as pd

with open("companies.txt") as file:
    lines = file.readlines()
    companyList = [line.rstrip() for line in lines]
stockData = {}
sum_df=pd.DataFrame()
for ticker in companyList:
    try:
        stockData[ticker] = si.get_data(ticker, start_date="01/01/2010", end_date="01/03/2020", interval="1mo")
        sum_df = sum_df.append(stockData[ticker])
    except:
        continue

sum_df.index=sum_df.index.rename("data")
sum_df.to_excel("output.xlsx") 

In [34]:
output = pd.read_excel("output.xlsx",index_col=None)
output.tail()

,date,open,high,low,close,adjclose,volume,ticker
57999,2019-09-01,290.570007,302.630005,289.269989,296.769989,285.776001,1303830000,SPY
58000,2019-10-01,297.739990,304.549988,284.820007,303.329987,293.441864,1386748300,SPY
58001,2019-11-01,304.920013,315.480011,304.739990,314.309998,304.063934,1037123500,SPY
58002,2019-12-01,314.589996,323.799988,307.130005,321.859985,311.367798,1285175800,SPY
58003,2020-01-01,323.540009,332.950012,320.359985,321.730011,312.772308,1392003800,SPY


In [35]:
output['monthly_return'] = output.groupby(['ticker'])['adjclose'].apply(pd.Series.pct_change)*100
output.tail()

,date,open,high,low,close,adjclose,volume,ticker,monthly_return
57999,2019-09-01,290.570007,302.630005,289.269989,296.769989,285.776001,1303830000,SPY,1.477179
58000,2019-10-01,297.739990,304.549988,284.820007,303.329987,293.441864,1386748300,SPY,2.682473
58001,2019-11-01,304.920013,315.480011,304.739990,314.309998,304.063934,1037123500,SPY,3.619821
58002,2019-12-01,314.589996,323.799988,307.130005,321.859985,311.367798,1285175800,SPY,2.402082
58003,2020-01-01,323.540009,332.950012,320.359985,321.730011,312.772308,1392003800,SPY,0.451078


Next, we add the company's sector into the final dataset as a new column by merging the constituents.csv file with our dataset.

In [36]:
spList = spList.rename(columns={'Symbol': 'ticker', 'Sector': 'sector', "Name": 'name'})
df2 = pd.DataFrame({'name': ["S&P 500"],'ticker': ['SPY'], 'sector': ['All']})
spList = spList.append(df2, ignore_index=True, sort=False)

In [37]:
finalOutput = pd.merge(output, spList, on="ticker")


check = (output.shape[0] == finalOutput.shape[0])

print("Was the join performed correctly? {}".format(check))

print(finalOutput.shape[0])
print(output.shape[0])

finalOutput.tail()

Was the join performed correctly? False
58246
58004


,date,open,high,low,close,adjclose,volume,ticker,monthly_return,name,sector
58241,2019-12-01,314.589996,323.799988,307.130005,321.859985,311.367798,1285175800,SPY,2.402082,S&P 500,All
58242,2019-12-01,314.589996,323.799988,307.130005,321.859985,311.367798,1285175800,SPY,2.402082,S&P 500,All
58243,2020-01-01,323.540009,332.950012,320.359985,321.730011,312.772308,1392003800,SPY,0.451078,S&P 500,All
58244,2020-01-01,323.540009,332.950012,320.359985,321.730011,312.772308,1392003800,SPY,0.451078,S&P 500,All
58245,2020-01-01,323.540009,332.950012,320.359985,321.730011,312.772308,1392003800,SPY,0.451078,S&P 500,All


Next, we made the 'date' column into a datetime object.

In [38]:
finalOutput['date']= pd.to_datetime(finalOutput['date'])

Now, we create three new columns: The first column, titled 'averages' calculates the average monthly return of a company. The next column tracks whether an observation has an above average monthly return for a company that month using the 'averages' column. The last column tracks whether the previous month's return was higher than the average monthly return. We then write this csv as our final analysis-ready dataset, titled 'finalOutput.csv'

In [39]:
outputNew = finalOutput
outputNew['month']=pd.DatetimeIndex(outputNew['date']).month
outputNew.head()
tickers = outputNew.ticker.unique()
averages = outputNew.groupby(['ticker']).monthly_return.mean()
for i in tickers:
    outputNew['averages'] = averages[i]
    
def isAboveAverage(averages,monthlyret):
    if monthlyret > averages:
        return 1
    return 0


outputNew["above_avg"] = outputNew.apply(lambda x: isAboveAverage(x["averages"],x['monthly_return']), axis=1)

outputNew.tail()

def previous_above_avg(dataframe):
    i=0
    while i < dataframe.shape[0]:
        startingTicker=dataframe.iloc[i]["ticker"]
        i+=1
        #Keep going until change in ticker or year
        while i < dataframe.shape[0] and dataframe.iloc[i]["ticker"] == startingTicker:
            dataframe.at[i,"prev_above_avg"] =  dataframe.at[i-1,"above_avg"]
            i+=1
    return dataframe

outputNew["prev_above_avg"]=None

outputNew=previous_above_avg(outputNew)
outputNew.to_csv('finalOutput.csv', index=False)
outputNew.head()

,date,open,high,low,close,adjclose,volume,ticker,monthly_return,name,sector,month,averages,above_avg,prev_above_avg
0,2010-01-01,26.000362,27.247843,25.376621,25.400612,19.294617,259719884,ABT,NaN,Abbott Laboratories,Health Care,1,1.152506,0,None
1,2010-02-01,25.482178,26.389000,25.271067,26.043545,19.927471,303364943,ABT,3.279954,Abbott Laboratories,Health Care,2,1.152506,1,0
2,2010-03-01,26.086725,26.652889,25.050358,25.275864,19.340067,402914340,ABT,-2.947711,Abbott Laboratories,Health Care,3,1.152506,0,1
3,2010-04-01,25.333441,25.549351,23.706919,24.546568,18.782045,361812837,ABT,-2.885313,Abbott Laboratories,Health Care,4,1.152506,0,0
4,2010-05-01,24.685711,24.752882,21.715748,22.819288,17.607489,436325761,ABT,-6.253615,Abbott Laboratories,Health Care,5,1.152506,0,0
